In [1]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import numpy as np
import random

Using TensorFlow backend.
C:\anaconda\envs\test3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\anaconda\envs\test3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\anaconda\envs\test3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\anaconda\envs\test3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Fu

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
X_train.shape

(60000, 28, 28)

In [4]:
X_train = (X_train/255).reshape(-1, 784)
X_test = (X_test/255).reshape(-1, 784)

In [10]:
from keras.layers.advanced_activations import LeakyReLU

g = Sequential()
g.add(Dense(units=256, input_dim=100, activation=LeakyReLU(0.2)))
g.add(Dense(units=512, activation=LeakyReLU(0.2)))
g.add(Dense(units=1024, activation=LeakyReLU(0.2)))
g.add(Dense(units=784, activation='sigmoid'))

g.compile(loss='binary_crossentropy', 
          optimizer='adam', 
          metrics=['accuracy'])

C:\anaconda\envs\test3\lib\site-packages\keras\activations.py:235: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
d = Sequential()
d.add(Dense(units=1024, input_dim=784, activation=LeakyReLU(0.2)))
d.add(Dense(units=512, activation=LeakyReLU(0.2)))
d.add(Dense(units=256, activation=LeakyReLU(0.2)))
d.add(Dense(units=1, activation='sigmoid'))

d.compile(loss='binary_crossentropy', 
          optimizer='adam', 
          metrics=['accuracy'])

d.trainable = False

In [12]:
from keras.models import Model
from keras.layers import Input

gan_input = Input(shape=(100,))
x = g(inputs=gan_input)
gan_output = d(x)

model = Model(gan_input, gan_output)

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [15]:
EPOCHS = 10
BATCH_SIZE = 128
num_of_batches = int(X_train.shape[0] / BATCH_SIZE)

for e in range(EPOCHS):
    for n in range(num_of_batches):
        noise = np.random.normal(0, 1, size=(BATCH_SIZE, 100))
        fake_data = g.predict(noise)
        
        real_data = X_train[np.random.randint(0, X_train.shape[0], size=BATCH_SIZE)]
        
        X = np.concatenate([real_data, fake_data])
        y = np.zeros(2*BATCH_SIZE)
        y[:BATCH_SIZE] = 0.9
        
        d.trainable = True
        d_loss = d.train_on_batch(X, y) # 판별자 학습
        
        d.trainable = False
        
        noise2 = np.random.normal(0, 1, size=(BATCH_SIZE, 100))
        y2 = np.ones(BATCH_SIZE)
        g_loss = model.train_on_batch(noise2, y2)
            
        print('EPOCH =', e+1, ', BATCH =', n+1, ', G_LOSS=', g_loss, ', D_LOSS =', d_loss)


EPOCH = 1 , BATCH = 1 , G_LOSS= [5.4816628, 0.0] , D_LOSS = [0.63528407, 0.5]
EPOCH = 1 , BATCH = 2 , G_LOSS= [3.603641, 0.0] , D_LOSS = [0.3201624, 0.5]
EPOCH = 1 , BATCH = 3 , G_LOSS= [6.1253796, 0.0] , D_LOSS = [0.21493411, 0.5]
EPOCH = 1 , BATCH = 4 , G_LOSS= [8.465549, 0.0] , D_LOSS = [0.21138571, 0.5]
EPOCH = 1 , BATCH = 5 , G_LOSS= [9.333723, 0.0] , D_LOSS = [0.20301726, 0.5]
EPOCH = 1 , BATCH = 6 , G_LOSS= [8.618339, 0.0] , D_LOSS = [0.20358337, 0.5]
EPOCH = 1 , BATCH = 7 , G_LOSS= [7.4304514, 0.0] , D_LOSS = [0.2587287, 0.5]
EPOCH = 1 , BATCH = 8 , G_LOSS= [5.213706, 0.0] , D_LOSS = [0.27844134, 0.5]
EPOCH = 1 , BATCH = 9 , G_LOSS= [5.953741, 0.0] , D_LOSS = [0.24401116, 0.5]
EPOCH = 1 , BATCH = 10 , G_LOSS= [7.1833467, 0.0] , D_LOSS = [0.20506708, 0.5]
EPOCH = 1 , BATCH = 11 , G_LOSS= [7.745524, 0.0] , D_LOSS = [0.18790978, 0.5]
EPOCH = 1 , BATCH = 12 , G_LOSS= [8.159167, 0.0] , D_LOSS = [0.17996433, 0.5]
EPOCH = 1 , BATCH = 13 , G_LOSS= [8.85416, 0.0] , D_LOSS = [0.1847616,

EPOCH = 1 , BATCH = 104 , G_LOSS= [11.643045, 0.0] , D_LOSS = [0.16298121, 0.5]
EPOCH = 1 , BATCH = 105 , G_LOSS= [11.69873, 0.0] , D_LOSS = [0.16322654, 0.5]
EPOCH = 1 , BATCH = 106 , G_LOSS= [11.783775, 0.0] , D_LOSS = [0.16303268, 0.5]
EPOCH = 1 , BATCH = 107 , G_LOSS= [11.819577, 0.0] , D_LOSS = [0.16293311, 0.5]
EPOCH = 1 , BATCH = 108 , G_LOSS= [11.699247, 0.0] , D_LOSS = [0.16306797, 0.5]
EPOCH = 1 , BATCH = 109 , G_LOSS= [11.623127, 0.0] , D_LOSS = [0.16297242, 0.5]
EPOCH = 1 , BATCH = 110 , G_LOSS= [11.733094, 0.0] , D_LOSS = [0.16316035, 0.5]
EPOCH = 1 , BATCH = 111 , G_LOSS= [11.803253, 0.0] , D_LOSS = [0.16309378, 0.5]
EPOCH = 1 , BATCH = 112 , G_LOSS= [11.704111, 0.0] , D_LOSS = [0.16300224, 0.5]
EPOCH = 1 , BATCH = 113 , G_LOSS= [11.665404, 0.0] , D_LOSS = [0.16281807, 0.5]
EPOCH = 1 , BATCH = 114 , G_LOSS= [11.727416, 0.0] , D_LOSS = [0.16311714, 0.5]
EPOCH = 1 , BATCH = 115 , G_LOSS= [11.788573, 0.0] , D_LOSS = [0.16307172, 0.5]
EPOCH = 1 , BATCH = 116 , G_LOSS= [11.677

EPOCH = 1 , BATCH = 208 , G_LOSS= [12.533377, 0.0] , D_LOSS = [0.1629598, 0.5]
EPOCH = 1 , BATCH = 209 , G_LOSS= [12.4801655, 0.0] , D_LOSS = [0.16293493, 0.5]
EPOCH = 1 , BATCH = 210 , G_LOSS= [12.328211, 0.0] , D_LOSS = [0.16274503, 0.5]
EPOCH = 1 , BATCH = 211 , G_LOSS= [12.557123, 0.0] , D_LOSS = [0.16311957, 0.5]
EPOCH = 1 , BATCH = 212 , G_LOSS= [12.532311, 0.0] , D_LOSS = [0.16293874, 0.5]
EPOCH = 1 , BATCH = 213 , G_LOSS= [12.382109, 0.0] , D_LOSS = [0.16275452, 0.5]
EPOCH = 1 , BATCH = 214 , G_LOSS= [12.531555, 0.0] , D_LOSS = [0.16299087, 0.5]
EPOCH = 1 , BATCH = 215 , G_LOSS= [12.607542, 0.0] , D_LOSS = [0.16279584, 0.5]
EPOCH = 1 , BATCH = 216 , G_LOSS= [12.487363, 0.0] , D_LOSS = [0.16280353, 0.5]
EPOCH = 1 , BATCH = 217 , G_LOSS= [12.52829, 0.0] , D_LOSS = [0.16281499, 0.5]
EPOCH = 1 , BATCH = 218 , G_LOSS= [12.626793, 0.0] , D_LOSS = [0.16276078, 0.5]
EPOCH = 1 , BATCH = 219 , G_LOSS= [12.556246, 0.0] , D_LOSS = [0.16281098, 0.5]
EPOCH = 1 , BATCH = 220 , G_LOSS= [12.517

EPOCH = 1 , BATCH = 312 , G_LOSS= [13.077903, 0.0] , D_LOSS = [0.16268554, 0.5]
EPOCH = 1 , BATCH = 313 , G_LOSS= [13.044954, 0.0] , D_LOSS = [0.16267264, 0.5]
EPOCH = 1 , BATCH = 314 , G_LOSS= [13.093191, 0.0] , D_LOSS = [0.1626578, 0.5]
EPOCH = 1 , BATCH = 315 , G_LOSS= [13.094576, 0.0] , D_LOSS = [0.16263884, 0.5]
EPOCH = 1 , BATCH = 316 , G_LOSS= [13.031076, 0.0] , D_LOSS = [0.1626643, 0.5]
EPOCH = 1 , BATCH = 317 , G_LOSS= [13.075217, 0.0] , D_LOSS = [0.16267154, 0.5]
EPOCH = 1 , BATCH = 318 , G_LOSS= [13.136076, 0.0] , D_LOSS = [0.16266443, 0.5]
EPOCH = 1 , BATCH = 319 , G_LOSS= [13.116478, 0.0] , D_LOSS = [0.16267014, 0.5]
EPOCH = 1 , BATCH = 320 , G_LOSS= [13.080507, 0.0] , D_LOSS = [0.16267172, 0.5]
EPOCH = 1 , BATCH = 321 , G_LOSS= [13.102775, 0.0] , D_LOSS = [0.16267428, 0.5]
EPOCH = 1 , BATCH = 322 , G_LOSS= [13.052612, 0.0] , D_LOSS = [0.16278537, 0.5]
EPOCH = 1 , BATCH = 323 , G_LOSS= [13.011415, 0.0] , D_LOSS = [0.16265522, 0.5]
EPOCH = 1 , BATCH = 324 , G_LOSS= [12.9166

EPOCH = 1 , BATCH = 415 , G_LOSS= [13.205932, 0.0] , D_LOSS = [0.16270599, 0.5]
EPOCH = 1 , BATCH = 416 , G_LOSS= [13.086262, 0.0] , D_LOSS = [0.16265191, 0.5]
EPOCH = 1 , BATCH = 417 , G_LOSS= [13.075829, 0.0] , D_LOSS = [0.16266678, 0.5]
EPOCH = 1 , BATCH = 418 , G_LOSS= [13.071403, 0.0] , D_LOSS = [0.16263331, 0.5]
EPOCH = 1 , BATCH = 419 , G_LOSS= [13.102151, 0.0] , D_LOSS = [0.16265729, 0.5]
EPOCH = 1 , BATCH = 420 , G_LOSS= [13.108816, 0.0] , D_LOSS = [0.16270164, 0.5]
EPOCH = 1 , BATCH = 421 , G_LOSS= [13.097263, 0.0] , D_LOSS = [0.1626381, 0.5]
EPOCH = 1 , BATCH = 422 , G_LOSS= [13.102233, 0.0] , D_LOSS = [0.16268264, 0.5]
EPOCH = 1 , BATCH = 423 , G_LOSS= [13.111809, 0.0] , D_LOSS = [0.16264275, 0.5]
EPOCH = 1 , BATCH = 424 , G_LOSS= [13.0737295, 0.0] , D_LOSS = [0.16264284, 0.5]
EPOCH = 1 , BATCH = 425 , G_LOSS= [13.104286, 0.0] , D_LOSS = [0.1626464, 0.5]
EPOCH = 1 , BATCH = 426 , G_LOSS= [13.072694, 0.0] , D_LOSS = [0.16263561, 0.5]
EPOCH = 1 , BATCH = 427 , G_LOSS= [13.138

...

EPOCH = 10 , BATCH = 42 , G_LOSS= [21.649384, 0.0] , D_LOSS = [0.17893487, 0.5]
EPOCH = 10 , BATCH = 43 , G_LOSS= [27.84535, 0.0] , D_LOSS = [0.1707399, 0.5]
EPOCH = 10 , BATCH = 44 , G_LOSS= [32.36133, 0.0] , D_LOSS = [0.16757882, 0.5]
EPOCH = 10 , BATCH = 45 , G_LOSS= [36.19708, 0.0] , D_LOSS = [0.17208865, 0.5]
EPOCH = 10 , BATCH = 46 , G_LOSS= [38.222164, 0.0] , D_LOSS = [0.17431197, 0.5]
EPOCH = 10 , BATCH = 47 , G_LOSS= [39.83589, 0.0] , D_LOSS = [0.1719458, 0.5]
EPOCH = 10 , BATCH = 48 , G_LOSS= [40.312508, 0.0] , D_LOSS = [0.17015433, 0.5]
EPOCH = 10 , BATCH = 49 , G_LOSS= [39.96704, 0.0] , D_LOSS = [0.16887845, 0.5]
EPOCH = 10 , BATCH = 50 , G_LOSS= [38.96714, 0.0] , D_LOSS = [0.17011842, 0.5]
EPOCH = 10 , BATCH = 51 , G_LOSS= [38.242825, 0.0] , D_LOSS = [0.17161751, 0.5]
EPOCH = 10 , BATCH = 52 , G_LOSS= [38.87178, 0.0] , D_LOSS = [0.17114758, 0.5]
EPOCH = 10 , BATCH = 53 , G_LOSS= [41.82556, 0.0] , D_LOSS = [0.171533, 0.5]
EPOCH = 10 , BATCH = 54 , G_LOSS= [45.065887, 0.0] ,

EPOCH = 10 , BATCH = 146 , G_LOSS= [25.036621, 0.0] , D_LOSS = [0.2410411, 0.5]
EPOCH = 10 , BATCH = 147 , G_LOSS= [31.854006, 0.0] , D_LOSS = [0.24616878, 0.5]
EPOCH = 10 , BATCH = 148 , G_LOSS= [38.094463, 0.0] , D_LOSS = [0.21473166, 0.5]
EPOCH = 10 , BATCH = 149 , G_LOSS= [43.07492, 0.0] , D_LOSS = [0.1912869, 0.5]
EPOCH = 10 , BATCH = 150 , G_LOSS= [39.987473, 0.0] , D_LOSS = [0.20634823, 0.5]
EPOCH = 10 , BATCH = 151 , G_LOSS= [35.232506, 0.0] , D_LOSS = [0.2304837, 0.5]
EPOCH = 10 , BATCH = 152 , G_LOSS= [30.994638, 0.0] , D_LOSS = [0.20194092, 0.5]
EPOCH = 10 , BATCH = 153 , G_LOSS= [25.917835, 0.0] , D_LOSS = [0.19066194, 0.5]
EPOCH = 10 , BATCH = 154 , G_LOSS= [15.229212, 0.0] , D_LOSS = [0.19891249, 0.5]
EPOCH = 10 , BATCH = 155 , G_LOSS= [13.414682, 0.0] , D_LOSS = [0.21178219, 0.5]
EPOCH = 10 , BATCH = 156 , G_LOSS= [12.732997, 0.0] , D_LOSS = [0.19578624, 0.5]
EPOCH = 10 , BATCH = 157 , G_LOSS= [11.899173, 0.0] , D_LOSS = [0.1800787, 0.5]
EPOCH = 10 , BATCH = 158 , G_LOSS

EPOCH = 10 , BATCH = 248 , G_LOSS= [8.590972, 0.0] , D_LOSS = [0.16651565, 0.5]
EPOCH = 10 , BATCH = 249 , G_LOSS= [8.87255, 0.0] , D_LOSS = [0.16555735, 0.5]
EPOCH = 10 , BATCH = 250 , G_LOSS= [7.6358757, 0.0] , D_LOSS = [0.16661546, 0.5]
EPOCH = 10 , BATCH = 251 , G_LOSS= [7.9822793, 0.0] , D_LOSS = [0.16524066, 0.5]
EPOCH = 10 , BATCH = 252 , G_LOSS= [7.530345, 0.0] , D_LOSS = [0.16655007, 0.5]
EPOCH = 10 , BATCH = 253 , G_LOSS= [7.602463, 0.0] , D_LOSS = [0.16644824, 0.5]
EPOCH = 10 , BATCH = 254 , G_LOSS= [7.8888507, 0.0] , D_LOSS = [0.16645515, 0.5]
EPOCH = 10 , BATCH = 255 , G_LOSS= [7.5229197, 0.0] , D_LOSS = [0.16678108, 0.5]
EPOCH = 10 , BATCH = 256 , G_LOSS= [7.9749904, 0.0] , D_LOSS = [0.16595364, 0.5]
EPOCH = 10 , BATCH = 257 , G_LOSS= [8.152887, 0.0] , D_LOSS = [0.16670807, 0.5]
EPOCH = 10 , BATCH = 258 , G_LOSS= [7.623873, 0.0] , D_LOSS = [0.16647775, 0.5]
EPOCH = 10 , BATCH = 259 , G_LOSS= [8.359836, 0.0078125] , D_LOSS = [0.16775313, 0.5]
EPOCH = 10 , BATCH = 260 , G_L

EPOCH = 10 , BATCH = 351 , G_LOSS= [12.905351, 0.0] , D_LOSS = [0.16447309, 0.5]
EPOCH = 10 , BATCH = 352 , G_LOSS= [12.981141, 0.0] , D_LOSS = [0.16403247, 0.5]
EPOCH = 10 , BATCH = 353 , G_LOSS= [12.853907, 0.0] , D_LOSS = [0.16415787, 0.5]
EPOCH = 10 , BATCH = 354 , G_LOSS= [12.575418, 0.0] , D_LOSS = [0.16455087, 0.5]
EPOCH = 10 , BATCH = 355 , G_LOSS= [12.146965, 0.0] , D_LOSS = [0.16454202, 0.5]
EPOCH = 10 , BATCH = 356 , G_LOSS= [11.594189, 0.0] , D_LOSS = [0.16411737, 0.5]
EPOCH = 10 , BATCH = 357 , G_LOSS= [10.720783, 0.0] , D_LOSS = [0.16448608, 0.5]
EPOCH = 10 , BATCH = 358 , G_LOSS= [10.611785, 0.0] , D_LOSS = [0.16459122, 0.5]
EPOCH = 10 , BATCH = 359 , G_LOSS= [10.742982, 0.0] , D_LOSS = [0.16467184, 0.5]
EPOCH = 10 , BATCH = 360 , G_LOSS= [10.957372, 0.0] , D_LOSS = [0.16476199, 0.5]
EPOCH = 10 , BATCH = 361 , G_LOSS= [11.082427, 0.0] , D_LOSS = [0.16391027, 0.5]
EPOCH = 10 , BATCH = 362 , G_LOSS= [11.049371, 0.0] , D_LOSS = [0.16463321, 0.5]
EPOCH = 10 , BATCH = 363 , G

EPOCH = 10 , BATCH = 454 , G_LOSS= [11.611691, 0.0] , D_LOSS = [0.16333106, 0.5]
EPOCH = 10 , BATCH = 455 , G_LOSS= [11.686027, 0.0] , D_LOSS = [0.16335094, 0.5]
EPOCH = 10 , BATCH = 456 , G_LOSS= [11.726786, 0.0] , D_LOSS = [0.1635516, 0.5]
EPOCH = 10 , BATCH = 457 , G_LOSS= [11.738913, 0.0] , D_LOSS = [0.1633938, 0.5]
EPOCH = 10 , BATCH = 458 , G_LOSS= [11.881615, 0.0] , D_LOSS = [0.163533, 0.5]
EPOCH = 10 , BATCH = 459 , G_LOSS= [11.888845, 0.0] , D_LOSS = [0.16351418, 0.5]
EPOCH = 10 , BATCH = 460 , G_LOSS= [11.848922, 0.0] , D_LOSS = [0.16344577, 0.5]
EPOCH = 10 , BATCH = 461 , G_LOSS= [13.563114, 0.0] , D_LOSS = [0.16358958, 0.5]
EPOCH = 10 , BATCH = 462 , G_LOSS= [11.917813, 0.0] , D_LOSS = [0.16338107, 0.5]
EPOCH = 10 , BATCH = 463 , G_LOSS= [12.109409, 0.0] , D_LOSS = [0.16316354, 0.5]
EPOCH = 10 , BATCH = 464 , G_LOSS= [12.2337675, 0.0] , D_LOSS = [0.16359654, 0.5]
EPOCH = 10 , BATCH = 465 , G_LOSS= [12.299644, 0.0] , D_LOSS = [0.16334033, 0.5]
EPOCH = 10 , BATCH = 466 , G_LO